# 1. Retrieve time series data and save as CSV

In this sample, we will focus on the following methods:

- Register signal definitions(\*) for general sensor data that convert binary data into numerical values.
- Retrieve data using registered signal definitions
- Convert the time series data to `pandas.DataFrame` and save it as a CSV file

```
A signal definition defines the process of cutting out a part of data formatted in binary format and reading it as value data such as numerical value and character string.
This process is done on the server side. See the intdash.Conversion derived class documentation for conversion types
(https://docs.intdash.jp/sdk/python/latest/en/signals.html#intdash.Conversion)
```

## Scenario
In this scenario, a time series data uploaded from iOS application **Visual M2M Motion** is used. The data will be downloaded and saved in CSV format in your local storage.

## Preparation
Before starting this scenario, prepare the following.

- An edge that performs measurement
- A measurement uploaded with Visual M2M Motion on iPhone (which contains general sensor data)
- Signal definitions for general sensor data


### Data to be used
In this scenario, the following data needs to be prepared on the server side.

| Data item | Data name that appears in this scenario |
|:---|:---|
| Edge to register time series data | sdk_edge1|
| Signal definitions(\*) | `sp_ACCX`, `sp_ACCY`, `sp_ACCZ`|

(\*) In this scenario, use the signal definition that converts the acceleration data of the smartphone to floating point values. For the registration method, check the following procedure.

### Create measurement data and upload time series data
Upload the data using **Visual M2M Motion**. After registering the measurement, use **Visual M2M Data Visualizer** to check that the new measurement has been created.

### (Note) Operation of Visual M2M Motion in this scenario
1. Start Visual M2M Motion on iPhone and sign in as `sdk_edge1`.
2. Enable Settings > Sensor > Save to server.
3. Return to the Main screen and tap ▶ in the center of the screen.
4. Confirm that the measurement is started
5. Tap ■ to end the measurement

Confirm that the new measurement is displayed in "Stored Data" of Visual M2M Data Visualizer.
Follow the procedure below to retrieve the time series data of this measurement and save it in CSV format.

<img src="https://github.com/aptpod/intdash-py-sample-codes/tree/master/img/img1.png">

### Import packages and create a client
For `url` given to ` intdash.Client`, specify the environment of the intdash server. For `edge_token`, specify the token issued for the edge you use.
(\* Login with `username` and `password` is also possible, but it is recommended to use edge token for continued use.)

In [4]:
import pandas as pd

import intdash
from intdash import  timeutils

###  Create Client

client = intdash.Client(
    url = "https://example.intdash.jp",
    edge_token = "your_token",
)

### Register the signal definitions
In this scenario, "general sensor type" data is used. This data is in intdash original format. You need to convert the data from smartphone sensor type to numeric type (Float type this time) on the server side.
In order to perform this conversion, it is necessary to register "signal definitions".
Use the following executable file to convert data from the smartphone sensor type to numerical values.

[create-signal-general-sensor.ipynb](../0_create-signals/create-signal-general-sensor.ipynb)

In this sample, only the conversion definition of "acceleration" is registered.

### Confirm that the signal definitions are registered
Confirm the registered signal definitions

In [2]:
signals = client.signals.list(label='sp')

In [3]:
for s in signals:
    print(s.label,  end=', ')

sp_ACCX, sp_ACCY, sp_ACCZ, 

The preparations are complete.

## Retrieve the edge used for measurement

In [4]:
edges = client.edges.list(name='sdk_edge1')
sdk_edge1 = edges[0]

In [5]:
sdk_edge1.name

'sdk_edge1'

## (Optional) Retrieve the measurement
If you want to search using the UUID of the measurement, retrieve the measurement information.
Specify the time using `list()` at the first retrieval.

In [7]:
ms = client.measurements.list(
    edge_uuid=sdk_edge1.uuid,
    start=timeutils.str2timestamp('2020-07-09 00:00:00+09:00'),
    end=timeutils.str2timestamp('2020-07-10 00:00:00+09:00')
)

In [8]:
# Because there is only one measurement associated with `sdk_edge1`, it is specified as follows.
m = ms[0]
print(m)

uuid: db43f826-03a4-44a1-84ca-518fe93c637e
name: 
description: 
edge_uuid: bbe7c17a-6edf-436f-9bc6-c9aad382280c
duration: 0 days 00:00:18.146069
ended: True
basetime: 2020-07-09 08:34:10.832607+00:00
basetime_type: ntp
processed_ratio: 1
protected: True
markers: []
created_at: 2020-07-09 08:34:11.214944+00:00
updated_at: 2020-07-09 08:34:35.376772+00:00


## Retrieve time series data
Use `client.data_points` to retrieve the time series data.
For ``labels`` , specify the label names of the signal definitions.

In [9]:
dps = client.data_points.list(
    edge_name='sdk_edge1',
    start=timeutils.str2timestamp('2020-07-09 00:00:00+09:00'),
    end=timeutils.str2timestamp('2020-07-10 00:00:00+09:00'),
    labels=['sp_ACCX', 'sp_ACCY', 'sp_ACCZ']
)

In [10]:
print(dps[0])

time: 2020-07-09T08:34:11.095032000Z
measurement_uuid: db43f826-03a4-44a1-84ca-518fe93c637e
data_type: 11
channel: 1
data_id: sp_ACCY
data_payload: b'\x07sp_ACCY\x7f\x9e\x06\x0c\x92>\xa5?'


## Convert list of DataPoints to a DataFrame

You can convert the data to a DataFrame which has columns for each `data_id` as follows.

In [19]:
from intdash import data

df = pd.DataFrame( [ {
     'time' : d.time,
       d.data_id : data.Float.from_payload(d.data_payload).value  # convert binary to numbers.
    }
    for d in dps
]).groupby("time").last()

df

,sp_ACCY,sp_ACCZ,sp_ACCX
time,,,
2020-07-09T08:34:11.095032000Z,0.041493,0.217996,0.048094
2020-07-09T08:34:11.115194000Z,0.123338,0.305914,0.038043
2020-07-09T08:34:11.135356000Z,0.046133,0.436974,-0.051388
2020-07-09T08:34:11.155517000Z,-0.034988,0.233272,0.102698
2020-07-09T08:34:11.175679000Z,-0.055907,-0.164569,0.047395
...,...,...,...
2020-07-09T08:34:14.361262000Z,0.506122,0.231640,-1.372483
2020-07-09T08:34:14.381424000Z,0.354834,-0.354698,-1.570656
2020-07-09T08:34:14.401586000Z,-0.216771,0.578869,-0.578583


## Save in CSV format
Convert to a CSV file using the feature of DataFrame.

In [20]:
df.to_csv('./sdk_sample.csv')

The saved file will look like as follows.

<img src="https://github.com/aptpod/intdash-py-sample-codes/tree/master/img/img2.png">